# 교차 검증과 그리드 서치

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 검증 세트

In [8]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [9]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [10]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [11]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [12]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [13]:
from sklearn.tree import DecisionTreeClassifier  # DecisionTreeClassifier를 sklearn 라이브러리에서 불러옵니다.

dt = DecisionTreeClassifier(random_state=42)  # DecisionTreeClassifier 모델을 생성하고, random_state를 42로 설정하여 결과의 재현성을 보장합니다.
dt.fit(sub_input, sub_target)  # 학습 데이터를 사용하여 모델을 학습시킵니다.

print(dt.score(sub_input, sub_target))  # 학습 데이터(sub_input)에 대해 모델의 정확도를 출력합니다.
print(dt.score(val_input, val_target))  # 검증 데이터(val_input)에 대해 모델의 정확도를 출력합니다.

0.9971133028626413
0.864423076923077


## 교차 검증

In [14]:
from sklearn.model_selection import cross_validate  # 교차 검증을 위한 cross_validate 함수를 sklearn 라이브러리에서 가져옵니다.
# cross_validate는 데이터를 여러 개의 폴드(fold)로 나누어 모델을 평가하는 방법으로, 모델의 일반화 성능을 더 정확하게 측정할 수 있습니다.

scores = cross_validate(dt, train_input, train_target)  # 결정 트리 모델(dt)을 학습 데이터(train_input)와 타겟(train_target)으로 교차 검증하여 점수를 계산합니다.
print(scores)  # 교차 검증 결과를 출력합니다.


{'fit_time': array([0.        , 0.0121212 , 0.00552106, 0.        , 0.011163  ]), 'score_time': array([0.        , 0.00102592, 0.        , 0.        , 0.00091529]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [15]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
from sklearn.model_selection import StratifiedKFold  # StratifiedKFold는 층화된 교차 검증을 위한 클래스로, 각 폴드에 동일한 클래스 비율이 유지되도록 데이터를 나누어줍니다.
# 이를 통해 데이터의 클래스 불균형이 있는 경우에도 보다 균등한 평가를 할 수 있습니다.

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())  # 결정 트리 모델(dt)을 학습 데이터로 층화된 교차 검증(StratifiedKFold)하여 점수를 계산합니다.
print(np.mean(scores['test_score']))  # 교차 검증에서 얻은 테스트 점수의 평균을 계산하여 출력합니다.


0.855300214703487


In [16]:
from sklearn.model_selection import StratifiedKFold  # StratifiedKFold는 데이터의 클래스 비율을 유지하면서 교차 검증을 수행하기 위한 클래스입니다.
# 이 클래스는 교차 검증 시 데이터셋의 불균형을 해결하여 보다 신뢰할 수 있는 평가를 가능하게 합니다.

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)  # 데이터를 10개의 폴드로 나누고, 데이터를 섞은 후(Shuffle) 동일한 결과를 얻기 위해 random_state를 42로 설정합니다.
scores = cross_validate(dt, train_input, train_target, cv=splitter)  # 결정 트리 모델(dt)을 StratifiedKFold로 설정된 교차 검증(splitter)을 사용하여 점수를 계산합니다.
print(np.mean(scores['test_score']))  # 교차 검증에서 얻은 테스트 점수의 평균을 계산하여 출력합니다.


0.8574181117533719


## 하이퍼파라미터 튜닝

In [18]:
from sklearn.model_selection import GridSearchCV  # GridSearchCV는 주어진 하이퍼파라미터 조합을 사용해 모델의 최적의 성능을 찾기 위한 도구입니다.
# 이 도구는 여러 하이퍼파라미터 값을 조합하여 모델을 학습시킨 후, 가장 좋은 성능을 내는 조합을 선택합니다.

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}  # 결정 트리의 'min_impurity_decrease' 하이퍼파라미터에 대해 실험할 값


In [19]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [20]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [21]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [22]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [23]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [24]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [25]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [26]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [27]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [28]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

In [30]:
from scipy.stats import uniform, randint  # SciPy 라이브러리의 uniform(균등 분포)과 randint(정수형 균등 분포) 함수들을 가져옵니다.

In [32]:
rgen = randint(0, 10)  # 0에서 10 사이의 임의의 정수를 생성하는 randint 객체를 생성합니다.
rgen.rvs(10)           # rgen 객체를 이용해 10개의 난수를 생성합니다. (rvs는 'random variates'를 의미)


array([9, 3, 7, 3, 6, 6, 6, 8, 8, 0], dtype=int64)

In [34]:
np.unique(rgen.rvs(1000), return_counts=True)  # 1000개의 난수를 생성한 후, 고유한 값들과 각 값의 발생 횟수를 반환합니다.

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([ 75,  88, 112,  90, 116, 108, 102, 108,  96, 105], dtype=int64))

In [36]:
ugen = uniform(0, 1)  # 0과 1 사이의 균등 분포를 가지는 랜덤 변수 분포 객체를 생성합니다.
ugen.rvs(10)  # 생성된 객체를 사용해 10개의 난수를 생성합니다.

array([0.95716139, 0.04180899, 0.04056277, 0.52591301, 0.95891508,
       0.10237681, 0.45717185, 0.89014927, 0.19511906, 0.86681636])

In [37]:
params = {
    'min_impurity_decrease': uniform(0.0001, 0.001),  # 0.0001에서 0.001 사이의 균등 분포에서 'min_impurity_decrease' 값을 랜덤으로 선택합니다.
    'max_depth': randint(20, 50),  # 20에서 50 사이의 정수 중 하나를 'max_depth' 값으로 랜덤하게 선택합니다.
    'min_samples_split': randint(2, 25),  # 2에서 25 사이의 정수 중 하나를 'min_samples_split' 값으로 랜덤하게 선택합니다.
    'min_samples_leaf': randint(1, 25),  # 1에서 25 사이의 정수 중 하나를 'min_samples_leaf' 값으로 랜덤하게 선택합니다.
}

In [38]:
from sklearn.model_selection import RandomizedSearchCV  # 모델 하이퍼파라미터를 무작위로 탐색하기 위해 RandomizedSearchCV를 가져옵니다.

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,  
                        n_iter=100, n_jobs=-1, random_state=42)  
# DecisionTreeClassifier를 사용해 모델을 생성하고, 
# 하이퍼파라미터 탐색을 위해 RandomizedSearchCV를 설정합니다.
# params: 탐색할 하이퍼파라미터 범위를 정의한 딕셔너리.
# n_iter=100: 100번의 무작위 조합을 시도합니다.
# n_jobs=-1: 모든 CPU 코어를 사용해 병렬로 작업을 수행합니다.
# random_state=42: 무작위성 제어를 위해 시드 값을 설정합니다.

gs.fit(train_input, train_target)  # 설정된 RandomizedSearchCV 객체를 사용해 train_input과 train_target 데이터를 학습시킵니다.


RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000027BD38F1E50>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000027BD38F0850>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000027BD38F36D0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000027BD38F1990>},
                   random_state=42)

In [39]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [40]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [41]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


## 확인문제

In [42]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000027BD38F1E50>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000027BD38F0850>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000027BD38F36D0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000027BD38F1990>},
                   random_state=42)

In [43]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
